In [33]:
import websocket
import json
import zlib
import time
import pickle
import datetime,os

try:
    import thread
except ImportError:
    import _thread as thread

import win32com.client
import re

import queue

In [34]:
websocket_url='wss://ks-live-dmcmt-bj6-pm-01.chat.bilibili.com/sub'

# 尤超白
# send1=[ 0x00, 0x00, 0x00, 0x6D, 0x00, 0x10, 0x00, 0x01, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00, 0x01, 0x7B, 0x22, 0x75, 0x69, 0x64, 0x22, 0x3A, 0x32, 0x38, 0x34, 0x39, 0x36, 0x34, 0x37, 0x37, 0x2C, 0x22, 0x72, 0x6F, 0x6F, 0x6D, 0x69, 0x64, 0x22, 0x3A, 0x31, 0x34, 0x37, 0x30, 0x37, 0x34, 0x39, 0x35, 0x2C, 0x22, 0x70, 0x72, 0x6F, 0x74, 0x6F, 0x76, 0x65, 0x72, 0x22, 0x3A, 0x32, 0x2C, 0x22, 0x70, 0x6C, 0x61, 0x74, 0x66, 0x6F, 0x72, 0x6D, 0x22, 0x3A, 0x22, 0x77, 0x65, 0x62, 0x22, 0x2C, 0x22, 0x63, 0x6C, 0x69, 0x65, 0x6E, 0x74, 0x76, 0x65, 0x72, 0x22, 0x3A, 0x22, 0x31, 0x2E, 0x36, 0x2E, 0x33, 0x22, 0x2C, 0x22, 0x74, 0x79, 0x70, 0x65, 0x22, 0x3A, 0x32, 0x7D ]
# 大野喵渣
send1=[ 0x00, 0x00, 0x00, 0x6C, 0x00, 0x10, 0x00, 0x01, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00, 0x01, 0x7B, 0x22, 0x75, 0x69, 0x64, 0x22, 0x3A, 0x32, 0x38, 0x34, 0x39, 0x36, 0x34, 0x37, 0x37, 0x2C, 0x22, 0x72, 0x6F, 0x6F, 0x6D, 0x69, 0x64, 0x22, 0x3A, 0x31, 0x38, 0x39, 0x39, 0x38, 0x34, 0x31, 0x2C, 0x22, 0x70, 0x72, 0x6F, 0x74, 0x6F, 0x76, 0x65, 0x72, 0x22, 0x3A, 0x32, 0x2C, 0x22, 0x70, 0x6C, 0x61, 0x74, 0x66, 0x6F, 0x72, 0x6D, 0x22, 0x3A, 0x22, 0x77, 0x65, 0x62, 0x22, 0x2C, 0x22, 0x63, 0x6C, 0x69, 0x65, 0x6E, 0x74, 0x76, 0x65, 0x72, 0x22, 0x3A, 0x22, 0x31, 0x2E, 0x36, 0x2E, 0x33, 0x22, 0x2C, 0x22, 0x74, 0x79, 0x70, 0x65, 0x22, 0x3A, 0x32, 0x7D ]
send2=[ 0x00, 0x00, 0x00, 0x1F, 0x00, 0x10, 0x00, 0x01, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x01, 0x5B, 0x6F, 0x62, 0x6A, 0x65, 0x63, 0x74, 0x20, 0x4F, 0x62, 0x6A, 0x65, 0x63, 0x74, 0x5D ]

In [35]:
user_gift_dir='D:/OneDrive/其他/直播相关/program/database/user_gift.pkl'
user_name_speak_dir='D:/OneDrive/其他/直播相关/program/database/user_name_speak.pkl'
log_dir='D:\OneDrive\其他\直播相关\program\log.txt'
def name_dir(i):
    return 'D:\OneDrive\其他\直播相关\program\\name_{}.txt'.format(i)

current_cat=1
total_cat=7

try:
    with open(user_gift_dir,'rb') as f:
        user_gift=pickle.load(f)
except:
    user_gift={}


In [36]:
def decode_json(msg):
    return json.loads(msg[16:])
def decomp(msg):
    try:
        return zlib.decompress(msg[16:])
    except Exception:
        print('error in decomp : {}'.format(msg[16:]))

def decode_msg(msg,debug=False):
    if debug:
        print(msg)
    if len(msg)==16:
        return []
    if len(msg)==20:
        print('当前观众：{}'.format(int.from_bytes(msg[16:], byteorder='big')))
        return []
    try:
        return [decode_json(msg)]
    except Exception:
        temp = decomp(msg)
#         print(split_jsons(temp))
        return [json.loads(i) for i in split_jsons(temp)]
    
def split_jsons(test_msg_4):
    activate=1
    stack=0
    start=[]
    end=[]
    escape=0
    i=16
    while i<len(test_msg_4):
#         print('i={},msg[i]={},stack={}'.format(i,bytes([test_msg_4[i]]),stack))
        if test_msg_4[i]==34 and escape%2!=1:
            activate=1-activate
            i+=1
            continue
        if test_msg_4[i]==92:
            escape+=1
        else:
            escape=0
        if activate:
            if test_msg_4[i]==123:
                if stack==0:
                    start.append(i)
                stack+=1
            if test_msg_4[i]==125:
                stack-=1
                if stack==0:
                    end.append(i)
                    i+=16
        i+=1
    result=[]
    for i2 in range(len(start)):
        try:
            result.append(test_msg_4[start[i2]:end[i2]+1])
        except Exception:
            print('error while append bytes to result at split_jsons.')
            print('start & end : {},{}'.format(start,end))
            print('msg : \n{}'.format(test_msg_4))
    return result

{'cmd': 'DANMU_MSG', 'info': [[0, 1, 25, 16777215, 1560431492, 1560431455, 0, 'd404101a', 0, 0, 0], '111', [28496477, '大野喵渣', 0, 0, 0, 10000, 1, ''], [9, '男友力', '尤超白', 14707495, 10512625, ''], [21, 0, 5805790, '>50000'], ['', ''], 0, 0, None, {'ts': 1560431492, 'ct': 'A567416F'}]}

{'cmd': 'SEND_GIFT', 'data': {'giftName': '辣条', 'num': 1, 'uname': '大野喵渣', 'face': 'http://i1.hdslb.com/bfs/face/8aa40398c14bf712f86687a04994186189b01314.jpg', 'guard_level': 0, 'rcost': 4259033, 'uid': 28496477, 'top_list': [], 'timestamp': 1560431512, 'giftId': 1, 'giftType': 0, 'action': '喂食', 'super': 0, 'super_gift_num': 0, 'price': 100, 'rnd': '1560431455', 'newMedal': 0, 'newTitle': 0, 'medal': [], 'title': '', 'beatId': '0', 'biz_source': 'live', 'metadata': '', 'remain': 6, 'gold': 0, 'silver': 0, 'eventScore': 0, 'eventNum': 0, 'smalltv_msg': [], 'specialGift': None, 'notice_msg': [], 'capsule': None, 'addFollow': 0, 'effect_block': 1, 'coin_type': 'silver', 'total_coin': 100, 'effect': 0, 'tag_image': '', 'user_count': 0}}

In [37]:
def handler(msg,debug=False):
    if debug:
        print(msg)
    if msg['cmd']=='SEND_GIFT':
        print('{}:{},{}'.format(msg['data']['uname'],msg['data']['giftName'],msg['data']['num']))
        total=msg['data']['num']
        try:
            total+=user_gift[(msg['data']['uid'],msg['data']['giftName'])]
        except:
            pass
        text='{}赠送了{}{}个，总数{}个。'.format(msg['data']['uname'],msg['data']['giftName'],msg['data']['num'],total)
        file=open(log_dir,'ab')
        file.write(text.encode('utf-8')+b'\n')
        file.close()
        user_gift[(msg['data']['uid'],msg['data']['giftName'])]=total
    if msg['cmd']=='DANMU_MSG':
        if msg['info'][1][:3]=='#抢猫':
            global current_cat
            file=open(name_dir(current_cat),'wb')
            file.write(msg['info'][2][1].encode('utf-8'))
            file.close()
            if current_cat==7:
                current_cat=1
            else:
                current_cat+=1
            

In [38]:
def on_message(ws, message):
    [handler(i,True) for i in decode_msg(message)]

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    def run(*args):
        ws.send(bytes(send1))
        while True:
            ws.send(bytes(send2))
            time.sleep(30)
    thread.start_new_thread(run, ())


if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp(websocket_url,
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

--- request header ---
GET /sub HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: ks-live-dmcmt-bj6-pm-01.chat.bilibili.com
Origin: http://ks-live-dmcmt-bj6-pm-01.chat.bilibili.com
Sec-WebSocket-Key: hweoj6gHVTnfcoXVDzZNqw==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Accept: Dv2AtNGAHw/ViqOB73wZT1J9nxQ=
-----------------------
send: b"\x81\xec\\a\x19>\\a\x19R\\q\x19?\\a\x199\\a\x19?'ClW8C#\x0cdU \x08hV.\x12~\x13vQ1\x08}\x1cfP!\x07eY-\x0fpCiL3\x15vH9\x13;\x04nM;N0\x00mX3\x13t\x1cfCn[>C5\x1c?\rp[2\x15o[.C#\x1cmO/\x10oC5\x1c(\x18i[~[+C"
send: b'\x81\x9f7\xffA\xc57\xffA\xda7\xefA\xc47\xffA\xc77\xffA\xc4l\x90#\xafR\x9c5\xe5x\x9d+\xa0T\x8b\x1c'


当前观众：1
{'cmd': 'DANMU_MSG', 'info': [[0, 1, 25, 16777215, 1560433337, 1560433082, 0, 'd404101a', 0, 0, 0], '111', [28496477, '大野喵渣', 0, 0, 0, 10000, 1, ''], [9, '男友力', '尤超白', 14707495, 10512625, ''], [21, 0, 5805790, '>50000'], ['', ''], 0, 0, None, {'ts': 1560433337, 'ct': '6F5D54B'}]}


Unhandled exception in thread started by <function on_open.<locals>.run at 0x000001C7A2F737B8>


WebSocketConnectionClosedException: Connection is already closed.

{'cmd': 'DANMU_MSG', 'info': [[0, 1, 25, 16777215, 1560433349, 1560433082, 0, 'd404101a', 0, 0, 0], '#抢猫', [28496477, '大野喵渣', 0, 0, 0, 10000, 1, ''], [9, '男友力', '尤超白', 14707495, 10512625, ''], [21, 0, 5805790, '>50000'], ['', ''], 0, 0, None, {'ts': 1560433349, 'ct': 'C11D71C'}]}


...

当前观众：1


send: b'\x81\x9f8G?c8G?|8W?b8G?a8G?bc(]\t]$KCw%U\x06[3b'


当前观众：1
{'cmd': 'SYS_MSG', 'msg': ' :?主播:?筱晴QAQ:?开启了“任意门”，点击前往TA的房间去抽奖吧！', 'msg_text': ' :?主播:?筱晴QAQ:?开启了“任意门”，点击前往TA的房间去抽奖吧！', 'msg_common': '全区广播：主播<%筱晴QAQ%>开启了“任意门”，点击前往TA的房间去抽奖吧！', 'msg_self': '全区广播：主播<%筱晴QAQ%>开启了“任意门”，快来抽奖吧！', 'rep': 1, 'styleType': 2, 'url': 'http://live.bilibili.com/10116204', 'roomid': 10116204, 'real_roomid': 10116204, 'rnd': 37399634, 'broadcast_type': 0}
{'cmd': 'NOTICE_MSG', 'full': {'head_icon': 'http://i0.hdslb.com/bfs/live/a3e164513bd030f1113a8fbe1165454880c3ee4c.webp', 'tail_icon': 'http://i0.hdslb.com/bfs/live/822da481fdaba986d738db5d8fd469ffa95a8fa1.webp', 'head_icon_fa': 'http://i0.hdslb.com/bfs/live/28c2f3dd68170391d173ca2efd02bdabc917df26.png', 'tail_icon_fa': 'http://i0.hdslb.com/bfs/live/38cb2a9f1209b16c0f15162b0b553e3b28d9f16f.png', 'head_icon_fan': 1, 'tail_icon_fan': 4, 'background': '#8675F5FF', 'color': '#FFFFFFFF', 'highlight': '#FDFF2FFF', 'time': 20}, 'half': {'head_icon': 'http://i0.hdslb.com/bfs/live/df00be5ee88c4cd53514090a58f8702368b9a

send: b'\x81\x9f8\x94\xdd\xe18\x94\xdd\xfe8\x84\xdd\xe08\x94\xdd\xe38\x94\xdd\xe0c\xfb\xbf\x8b]\xf7\xa9\xc1w\xf6\xb7\x84[\xe0\x80'
send: b'\x88\x82P\x88bNS`'



### closed ###


In [ ]:
with open(user_gift_dir,'wb') as f:
    pickle.dump(user_gift,f)